# assessment once only

- Assessment section must appear only once in the sample to make one start position and one end position
- Task 4 and 5 finetuned to find start, and end position of assessment section
- Assessment section should appear only once in one visited document, but sometimes the same assessment is written twice, so this code was written to filter it out.

In [ ]:
import glob

### section names

In [ ]:
section_types_paths = ["./data/02_type_to_file_links/sections_visits_2011to2020_task4.txt"]

section_types = {}
for i in range(len(section_types_paths)):
    section_types_path = section_types_paths[i]
    file = open(section_types_path, "r")
    lines = file.readlines()
    for l in range(len(lines)):
        if lines[l]=="\n":
            continue
        doctype = lines[l].split("\t")[0].strip("\n")
        section = lines[l].split("\t")[1].strip("\n")    
        key = doctype+"\t"+section

        val = lines[l].split("\t")[2].strip("\n")
        
        if "assessment" in val.lower():
            section_types[key] = "assessment"
        else:
            section_types[key] = "else"
                
    file.close()
# print(section_types)


In [ ]:
datapaths   = ["./data/04_sampled/train/*.txt", "./data/04_sampled/test/*.txt"]
output_path = ["./data/05_sampled/train", "./data/05_sampled/test"]

In [ ]:
def assessment_only_one(filepath):
    file = open(filepath, "r")
    lines = file.readlines()
    file.close()
    
    documents = {}
    
    doc_id_last = ""
    
    # split documents
    stacked = []
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        
        doc_id = line.split("\t")[0]
        if doc_id_last!=doc_id and l!=0:
            documents[doc_id_last] = stacked
            stacked = []
        
        stacked.append(line)
        doc_id_last = doc_id
    
    if len(stacked)>0:
        documents[doc_id_last] = stacked
        stacked = []
    
    
    # assessment
    new_documents = []
    for key in documents:
        document = documents[key]
        
        # asessment 가 등장하지 않은 문서는 모두 제거한다
        assessment_flag = False
        for d in range(len(document)):
            doc_id = document[d].split("\t")[0]
            sent_id = document[d].split("\t")[1]
            doctype = document[d].split("\t")[2]
            section = document[d].split("\t")[3]
            content = document[d].split("\t")[4]
            label = section_types[doctype+"\t"+section]
            if label=="assessment":
                assessment_flag=True
                break
        # 해당 문서 제거
        if assessment_flag==False:
            continue
            
        assessment_flag = False
        label_last = ""
        for d in range(len(document)):
            doc_id = document[d].split("\t")[0]
            sent_id = document[d].split("\t")[1]
            doctype = document[d].split("\t")[2]
            section = document[d].split("\t")[3]
            content = document[d].split("\t")[4]
            
            label = section_types[doctype+"\t"+section]
#             print("label: ", label)
            
            if label=="assessment" and label_last=="else":
                if assessment_flag==True:
                    break
            
            if label=="assessment":
                assessment_flag=True
            
            new_documents.append(document[d])
            
            label_last = label

    return new_documents

In [ ]:
import os

for d in range(len(datapaths)):
    filepaths = glob.glob(datapaths[d])
    filepaths.sort()
    
    if not os.path.exists(output_path[d]):
        os.makedirs(output_path[d])
    
    for f in range(len(filepaths)):
        filename = filepaths[f].split("/")[-1]
        print("filename: ", filename)
        new_documents = assessment_only_one(filepaths[f])
        if len(new_documents)>0:
            file = open(output_path[d]+"/"+filename, "w")
            file.write("\n".join(new_documents))
            file.close()
